# Resampling observational noise

First we need some imports.

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import numpy as np
from scipy import stats
import pylab as plt
import torch
import torchist
import swyft

## Training data

Now we generate training data.  As simple example, we consider the model

$$
x = z + \epsilon
$$

where the parameter $z \sim \mathcal{N}(\mu = 0, \sigma = 1)$ is standard normal distributed, and $\epsilon \sim \mathcal{N}(\mu = 0, \sigma = 0.1)$ is a small noise contribution.  We are interested in the posterior of $z$ given a measurement of parameter $x$.

In [13]:
class Simulator(swyft.Simulator):
    def __init__(self):
        super().__init__()
        self.transform_samples = swyft.to_numpy32

    def build(self, graph):
        z = graph.node('z', lambda: np.random.rand(1))
        x = graph.node('x', lambda z: z + np.random.randn(1)*0.1, z)
        
sim = Simulator()

In [14]:
samples = sim.sample(500)

100%|██████████| 500/500 [00:00<00:00, 30072.30it/s]


In [15]:
resampler = sim.get_resampler(targets = ["x"])

In [16]:
class Network(swyft.SwyftModule):
    def __init__(self):
        super().__init__()
        self.logratios = swyft.LogRatioEstimator_1dim(num_features = 1, num_params = 1, varnames = 'z')

    def forward(self, A, B):
        logratios = self.logratios(A['x'], B['z'])
        return logratios

## Trainer

Training is now done using the `SwyftTrainer` class, which extends `pytorch_lightning.Trainer` by methods like `infer` (see below).

In [17]:
trainer = swyft.SwyftTrainer(accelerator = 'gpu', max_epochs = 20, precision = 32)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


The `swyft.Samples` class provides convenience functions to generate data loaders for training and validation data.

In [18]:
dl_train = samples[:-100].get_dataloader(batch_size = 16, shuffle = True, on_after_load_sample = resampler)
dl_valid = samples[-100:].get_dataloader(batch_size = 16)

In [19]:
network = Network()

In [20]:
trainer.fit(network, dl_train, dl_valid)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params
-----------------------------------------------------
0 | logratios | LogRatioEstimator_1dim | 17.4 K
-----------------------------------------------------
17.4 K    Trainable params
0         Non-trainable params
17.4 K    Total params
0.070     Total estimated model params size (MB)


Epoch 0:  78%|███████▊  | 25/32 [00:00<00:00, 107.47it/s, loss=-0.512, v_num=1e+7] 
Validation: 0it [00:00, ?it/s]
Epoch 1:  78%|███████▊  | 25/32 [00:00<00:00, 108.05it/s, loss=-0.548, v_num=1e+7, val_loss=-.0171]
Validation: 0it [00:00, ?it/s]
Epoch 2:  78%|███████▊  | 25/32 [00:00<00:00, 104.21it/s, loss=-0.526, v_num=1e+7, val_loss=-.297]
Validation: 0it [00:00, ?it/s]
Epoch 3:  78%|███████▊  | 25/32 [00:00<00:00, 103.67it/s, loss=-0.589, v_num=1e+7, val_loss=-.406]
Validation: 0it [00:00, ?it/s]
Epoch 4:  78%|███████▊  | 25/32 [00:00<00:00, 105.47it/s, loss=-0.57, v_num=1e+7, val_loss=-.429] 
Validation: 0it [00:00, ?it/s]
Epoch 5:  78%|███████▊  | 25/32 [00:00<00:00, 96.12it/s, loss=-0.548, v_num=1e+7, val_loss=-.472]
Validation: 0it [00:00, ?it/s]
Epoch 6:  78%|███████▊  | 25/32 [00:00<00:00, 102.43it/s, loss=-0.519, v_num=1e+7, val_loss=-.439]
Validation: 0it [00:00, ?it/s]
Epoch 7:  78%|███████▊  | 25/32 [00:00<00:00, 110.71it/s, loss=-0.594, v_num=1e+7, val_loss=-.464]
Valida

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 32/32 [00:00<00:00, 96.48it/s, loss=-0.66, v_num=1e+7, val_loss=-.427] 
